In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
baseurl="https://www.truecar.com/used-cars-for-sale/listings/location-santa-clara-ca/?page="
page=['?page=1','?page=2']
CarList=[]
for i in range(1,81):
    driver.get(baseurl+str(i))
    car=[]
    spans=driver.find_elements_by_xpath('//a[@class="_abw475"]')
    for j in spans[:30]:
        car.append(j.text)
    details=[]
    ul= driver.find_elements_by_xpath('//ul[@class="d-inline-flex"]')
    for i in ul[:30]:
        details.append(i.text)
    price=[]
    p=driver.find_elements_by_xpath('//h4[@class="heading-2"]')
    for i in p[:30]:
        price.append(i.text)
    price_details=[m+"; "+n for m,n in zip(price,details)]
    price_details = list(map(lambda x: x.replace("\n","; "),price_details))
    car_details=[m+"; "+n for m,n in zip(car,price_details)]
    car_details = list(map(lambda x: x.replace("\n","; "),car_details))
    nextPage=[]
    for my_href in spans[:30]:
        nextPage.append(my_href.get_attribute("href"))
    vin=[]
    for i in nextPage:
        num=i.split('/')
        num[5]
        vin.append(num[5])
    car_info=[m+"; "+n for m,n in zip(car_details,vin)]
    car_info = list(map(lambda x: x.replace("\n","; "),car_info))
    #car_info
    car_info = list(map(lambda x: x.replace("Mileage: ","").replace(" miles","")
                           .replace(" Location:","").replace(" Exterior:","").replace(" Interior:","")
                           .replace(" VIN:","").replace(", CA","").replace(" $",""),car_info))
    df = pd.DataFrame([sub.split(";") for sub in car_info],columns=['Model','Price','Miles','Location','Distance','VIN'])
    attr=[]
    vin1=pd.Series(vin)
    for i in nextPage:
        driver.get(i)
        MPG=driver.find_elements_by_xpath('//span[contains(text(),"MPG") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        Transmission=driver.find_elements_by_xpath('//span[contains(text(),"Transmission") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        Engine=driver.find_elements_by_xpath('//span[contains(text(),"Engine") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        DriveType=driver.find_elements_by_xpath('//span[contains(text(),"Drive Type") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        FuelType=driver.find_elements_by_xpath('//span[contains(text(),"Fuel Type") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        Trim=driver.find_elements_by_xpath('//span[contains(text(),"Trim") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        CarFax=driver.find_elements_by_xpath('//span[@class="not-null"]')[0].text
        Exterior=driver.find_elements_by_xpath('//span[contains(text(),"Exterior Color") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text
        Interior=driver.find_elements_by_xpath('//span[contains(text(),"Interior") and @class="emphasized-feature-title"]/following-sibling::span[@class="emphasized-feature-description"]')[0].text

        new = [MPG,Transmission,Engine,DriveType,FuelType,Trim,CarFax,Exterior,Interior]
        attr.append(new)
        time.sleep(0.5)
    attr_df = pd.DataFrame(attr)
    attr_df.columns = ['Mpg', 'Transmission', 'Engine', 'DriveType', 'FuelType','Trim','CarFax','Exterior','Interior']
    attr_df['VIN']=vin1
    df['VIN'] = df['VIN'].str.strip()
    attr_df['VIN'] = attr_df['VIN'].str.strip()
    final_df = pd.merge(df,attr_df)
    CarList.append(final_df)
CarFinal=pd.concat(CarList,ignore_index=True)
CarFinal.to_csv('cars1_stats.csv', sep='\t')


